![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [120]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
print(ds_jobs.head())
ds_jobs.info()
ds_jobs["enrolled_university"].value_counts()


   student_id      city  ...  training_hours job_change
0        8949  city_103  ...              36        1.0
1       29725   city_40  ...              47        0.0
2       11561   city_21  ...              83        0.0
3       33241  city_115  ...              52        1.0
4         666  city_162  ...               8        0.0

[5 rows x 14 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8

no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64

In [121]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()


In [122]:
# Start coding here. Use as many cells as you like!
# Turn relevant_experience and job_change into dummies
ds_jobs_transformed=pd.get_dummies(ds_jobs_transformed, columns=["relevant_experience", "job_change"], dtype=bool)
ds_jobs_transformed=ds_jobs_transformed.drop(["relevant_experience_No relevant experience", "job_change_0.0"], axis=1)
ds_jobs_transformed.rename(columns={"relevant_experience_Has relevant experience":"relevant_experience", "job_change_1.0":"job_change"}, inplace=True)
print(ds_jobs_transformed.head())


   student_id      city  ...  relevant_experience job_change
0        8949  city_103  ...                 True       True
1       29725   city_40  ...                False      False
2       11561   city_21  ...                False      False
3       33241  city_115  ...                False       True
4         666  city_162  ...                 True      False

[5 rows x 14 columns]


In [123]:
#Store student_id and training_hours as int32
import numpy as np

ds_jobs_transformed["student_id"]= ds_jobs_transformed["student_id"].astype(np.int32)
ds_jobs_transformed["training_hours"]= ds_jobs_transformed["training_hours"].astype(np.int32)

#convert city_development_index to float16 
ds_jobs_transformed["city_development_index"] = ds_jobs_transformed["city_development_index"].astype(np.float16)

#convert city, gender, major_discipline, company_type  to category data type
#convert enrolled_university, education_level, experience, company_size, last_new_job   to ordered category type

list_to_category= ["city", "company_size", "company_type", "gender", "enrolled_university", "education_level", "experience", "last_new_job","major_discipline"]

ds_jobs_transformed[list_to_category]=ds_jobs_transformed[list_to_category].astype("category")


ds_jobs_transformed["enrolled_university"].cat.reorder_categories(new_categories=["Full time course", "Part time course", "no_enrollment"], ordered=True, inplace=True)

ds_jobs_transformed["education_level"].cat.reorder_categories(new_categories=["Phd", "Graduate", "Masters", "High School", "Primary School"], ordered=True, inplace=True)
print(ds_jobs_transformed["education_level"].cat.categories)


print(ds_jobs["experience"].value_counts())
ds_jobs_transformed["experience"].cat.reorder_categories(new_categories=["<1", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", ">20"], ordered=True, inplace=True)
print(ds_jobs_transformed["experience"].cat.categories)

print(ds_jobs["company_size"].value_counts())
ds_jobs_transformed["company_size"].cat.reorder_categories(new_categories=["<10", "10-49", "50-99", "100-499", "500-999", "1000-4999", "5000-9999", "10000+"], ordered=True, inplace=True)
print(ds_jobs_transformed["company_size"].cat.categories)

print(ds_jobs["last_new_job"].value_counts())
ds_jobs_transformed["last_new_job"].cat.reorder_categories(new_categories=["never", "1", "2", "3", "4", ">4"], ordered=True, inplace=True)
print(ds_jobs_transformed["last_new_job"].cat.categories)

Index(['Phd', 'Graduate', 'Masters', 'High School', 'Primary School'], dtype='object')
>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: experience, dtype: int64
Index(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '>20'],
      dtype='object')
50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: company_size, dtype: int64
Index(['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999',
       '10000+'],
      dtype='object')
1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
Name: last_new_job, dtype: int64
Index(['never', '1', '2', '3', '4', '>4'], 

In [124]:
#Filter data to students with 10+ years of experience at companies with 1000+ employees

filter_condition= (ds_jobs_transformed["experience"] >= "10") & (ds_jobs_transformed["company_size"]>="1000-4999")

ds_jobs_transformed= ds_jobs_transformed[filter_condition==True]


# check storage
print(ds_jobs.info())
print(ds_jobs.memory_usage())
print(ds_jobs_transformed.info())
print(ds_jobs_transformed.memory_usage())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me